<a href="https://www.kaggle.com/code/skyelermcq/jeopardy-generator?scriptVersionId=100129500" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#loading packages 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from google.cloud import bigquery # to do bigsql
from pandasql import sqldf #this is for doing sql 
from random import randint #to do random integers
from IPython.display import HTML #so I can download as a csv
import csv #so I can download as a csv
import base64 #so I can download as a csv
import io #writing csv to string
 

import os
pd.options.display.max_rows = 4000 #so I can print long df

In [2]:
# Reading the CSV file and saving as jeopardydb
pd.read_csv('/kaggle/input/200000-jeopardy-questions/JEOPARDY_CSV.csv', header='infer', keep_default_na=True, skip_blank_lines=True)
jeopardydb = pd.read_csv('/kaggle/input/200000-jeopardy-questions/JEOPARDY_CSV.csv', header='infer', keep_default_na=True, skip_blank_lines=True)


In [3]:


mysql = lambda q: sqldf(q, globals())

#This query gets all the single jeopardy categories
query1 = """ SELECT DISTINCT [ Category]
            FROM
                jeopardydb
            WHERE [ Round]='Jeopardy!';
"""



In [4]:


#This query gets all the double jeopardy categories
query2 = """ SELECT DISTINCT [ Category]
            FROM
                jeopardydb
            WHERE [ Round]='Double Jeopardy!';
"""



In [5]:
mysql = lambda q: sqldf(q, globals())

#This query gets all the final jeopardy categories
queryf = """ SELECT DISTINCT [ Category]
            FROM
                jeopardydb
                WHERE
            [ Round]='Final Jeopardy!';
"""


In [6]:
#So, we need five categories of single jeopardy

#initialize data frame
categ=mysql(query1) #gets a single jeopardy category
category = categ.loc[randint(0,categ.size)]
query = "SELECT * FROM jeopardydb WHERE [ Category]='"+category[0].replace(r"'", r"''")+"' AND [ Round]='Jeopardy!' LIMIT 5;"
jeoptable = mysql(query)

for i in range(4):    
    temptable = pd.DataFrame() #initialize dataframe
    while len(temptable)<5:
        categ=mysql(query1) #gets a single jeopardy category
        category = categ.loc[randint(0,categ.size)]
        query = "SELECT * FROM jeopardydb WHERE [ Category]='"+category[0].replace(r"'", r"''")+"' AND [ Round]='Jeopardy!' LIMIT 5;"
        temptable = mysql(query)
    temptable = temptable.rename(index=(lambda x : x + 5*(i+1)))
    jeoptable = jeoptable.append(temptable)
    
for i in range(5): 
    temptable = pd.DataFrame() #initialize dataframe
    while len(temptable)<5:
        categ=mysql(query2) #gets a double jeopardy category
        category = categ.loc[randint(0,categ.size)]
        query = "SELECT * FROM jeopardydb WHERE [ Category]='"+category[0].replace(r"'", r"''")+"' AND [ Round]='Double Jeopardy!' LIMIT 5;"
        temptable = mysql(query)
    temptable = temptable.rename(index=(lambda x : 25 + x + 5*i))
    jeoptable = jeoptable.append(temptable)
    

categ=mysql(queryf) #gets a final jeopardy category
category = categ.loc[randint(0,categ.size)]
query = "SELECT * FROM jeopardydb WHERE [ Category]='"+category[0].replace(r"'", r"''")+"' AND [ Round]='Final Jeopardy!' LIMIT 1;"
temptable = mysql(query)
temptable = temptable.rename(index=(lambda x : 50 + x))
jeoptable = jeoptable.append(temptable)
    
print(jeoptable)

    Show Number    Air Date             Round                       Category  \
0          3484  1999-10-28         Jeopardy!                   MOVIE SERIES   
1          3484  1999-10-28         Jeopardy!                   MOVIE SERIES   
2          3484  1999-10-28         Jeopardy!                   MOVIE SERIES   
3          3484  1999-10-28         Jeopardy!                   MOVIE SERIES   
4          3484  1999-10-28         Jeopardy!                   MOVIE SERIES   
5          5857  2010-02-16         Jeopardy!               SEWARD'S FOLLIES   
6          5857  2010-02-16         Jeopardy!               SEWARD'S FOLLIES   
7          5857  2010-02-16         Jeopardy!               SEWARD'S FOLLIES   
8          5857  2010-02-16         Jeopardy!               SEWARD'S FOLLIES   
9          5857  2010-02-16         Jeopardy!               SEWARD'S FOLLIES   
10         5826  2010-01-04         Jeopardy!                     RHETT-ORIC   
11         5826  2010-01-04         Jeop

In [7]:
#Now I will download to CSV

def use_mtxtemplate(df):
    output = io.StringIO()
    writer = csv.writer(output)
    
    #write blank lines
    for i in range(6):
        writer.writerow(['']*7) 
        
    writer.writerow(['',"CATEGORIES:"]+[df.at[i*5, ' Category'] for i in range(5)])
    writer.writerow(['200',"Clue"]+[df.at[i*5, ' Question'] for i in range(5)])
    writer.writerow(['',"Response"]+[df.at[i*5, ' Answer'] for i in range(5)])
    
    writer.writerow(['400',"Clue"]+[df.at[i*5+1, ' Question'] for i in range(5)])
    writer.writerow(['',"Response"]+[df.at[i*5+1, ' Answer'] for i in range(5)])
    
    writer.writerow(['600',"Clue"]+[df.at[i*5+2, ' Question'] for i in range(5)])
    writer.writerow(['',"Response"]+[df.at[i*5+2, ' Answer'] for i in range(5)])
    
    writer.writerow(['800',"Clue"]+[df.at[i*5+3, ' Question'] for i in range(5)])
    writer.writerow(['',"Response"]+[df.at[i*5+3, ' Answer'] for i in range(5)])
    
    writer.writerow(['1000',"Clue"]+[df.at[i*5+4, ' Question'] for i in range(5)])
    writer.writerow(['',"Response"]+[df.at[i*5+4, ' Answer'] for i in range(5)])
    
    #write blank lines
    for i in range(5):
        writer.writerow(['']*7) 
    
    writer.writerow(['',"CATEGORIES:"]+[df.at[i*5+25, ' Category'] for i in range(5)])
    writer.writerow(['200',"Clue"]+[df.at[i*5+25, ' Question'] for i in range(5)])
    writer.writerow(['',"Response"]+[df.at[i*5+25, ' Answer'] for i in range(5)])
    
    writer.writerow(['400',"Clue"]+[df.at[i*5+26, ' Question'] for i in range(5)])
    writer.writerow(['',"Response"]+[df.at[i*5+26, ' Answer'] for i in range(5)])
    
    writer.writerow(['600',"Clue"]+[df.at[i*5+27, ' Question'] for i in range(5)])
    writer.writerow(['',"Response"]+[df.at[i*5+27, ' Answer'] for i in range(5)])
    
    writer.writerow(['800',"Clue"]+[df.at[i*5+28, ' Question'] for i in range(5)])
    writer.writerow(['',"Response"]+[df.at[i*5+28, ' Answer'] for i in range(5)])
    
    writer.writerow(['1000',"Clue"]+[df.at[i*5+29, ' Question'] for i in range(5)])
    writer.writerow(['',"Response"]+[df.at[i*5+29, ' Answer'] for i in range(5)])
    
    #write blank lines
    for i in range(5):
        writer.writerow(['']*7) 
        
    writer.writerow(['Category:',df.at[50,' Category'],'','','','',''])
    writer.writerow(['Clue:',df.at[50,' Question'],'','','','',''])
    writer.writerow(['Prefix:','Who/what/when/where is','','','','',''])
    writer.writerow(['Response:',df.at[50,' Answer'],'','','','',''])

    return output.getvalue()

def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = use_mtxtemplate(df) #make our cute function to use Michael's template
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(jeoptable,"Jeopardy Table","forMichael.csv")